# Estadísticas del etiquetado de emociones

In [53]:
import pandas as pd
import numpy as np
import plotly.express as px
import json

In [54]:
with open('./jazmin_flores_11_08_21.json') as json_file:
    jazmin = json.load(json_file)
    
with open('./mara_vazquez_11_08_21.json') as json_file:
    mara = json.load(json_file)
    
with open('./omar_hernandez_11_08_21.json') as json_file:
    omar = json.load(json_file)

In [55]:
psicologos = {'jazmin':jazmin, 'mara':mara, 'omar':omar}

## Filtro un rango de fechas

In [56]:
# formato YYYY-MM-DD
start_date = datetime.date(2021, 8, 1)
end_date = datetime.date(2021, 8, 20)

In [57]:
# hago un diccionario que contenga 1 df por cada psicólogo
df_dict = {}

In [74]:
for k in psicologos.keys():

    data = psicologos[k]
    df = pd.DataFrame(columns = ['id', 'date', 'emotion', 'intensity', 'confidence', 'updated', 'channel', 'lead_time', 'time_interval'])

    i = 0
    for d in data:
        identity = d['id']
        channel = int(d['data']['url'][-10])
        lead_time = d['annotations'][0]['lead_time']

        # para calcular el campo 'updated'
        created_str = d['annotations'][0]['created_at'].replace('T', ' ')[:-8]
        updated_str = d['annotations'][0]['updated_at'].replace('T', ' ')[:-8]
        created_time = datetime.datetime.strptime(created_str, format)
        updated_time = datetime.datetime.strptime(updated_str, format)
        time_interval = (updated_time - created_time).total_seconds()
        
        # primero filtro por fecha
        date = created_time.date()
        
        # si no está en el rango, la paso de largo
        if date < start_date or date > end_date:
#             print('fuera_de_rango', date, start_date, end_date)
            continue

        updated = 0
        if time_interval>0:
            updated = 1

        # completo las columnas
        df.loc[i, 'id'] = identity
        df.loc[i, 'updated'] = updated
        df.loc[i, 'date'] = date
        df.loc[i, 'channel'] = channel
        df.loc[i, 'lead_time'] = lead_time
        df.loc[i, 'time_interval'] = time_interval

        # completo los 3 campos restantes
        results = d['annotations'][0]['result']
        for r in results:
            field_name = r['from_name']
            df.loc[i, field_name] = r['value']['choices'][0]

        i = i+1
        
    df_dict[k] = df

## 1) Del total de los audios de una semana, en cuántos hubo updates?

In [65]:
update_ratio = {} # ratio cantidad de updates sobre cantidad de etiquetas
n_labels = {} # cuántos audios etiquetó

for k in psicologos.keys():
    
#     print(k)
#     print(df.head())
    
    df = df_dict[k]
    
    n_updates = df['updated'].sum() # cuántos updates hizo la persona
    N = df.shape[0] # cuántos audios etiquetó
    
    update_ratio[k] = n_updates/float(N)
    n_labels[k] = N

fig = px.histogram(x=update_ratio.keys(), y=update_ratio.values(), title="Ratio de updates sobre cantidad de audios", width=600, height=400).update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Psicólogo", yaxis_title="Ratio")
fig.show()

## 2) Cuántas etiquetas por semana hubo?

In [60]:
fig = px.histogram(x=n_labels.keys(), y=n_labels.values(), title="Cantidad de etiquetas por persona en la semana", width=600, height=400).update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Psicólogo", yaxis_title="Cantidad de etiquetas")
fig.show()

## 3) Distribución de confianza por persona

In [61]:
for k in psicologos.keys():
    df = df_dict[k]
    
    df_agg = df[['id', 'confidence']].groupby(['confidence']).count()
    df_agg.reset_index(inplace=True)

    fig = px.histogram(df_agg, x=df_agg['confidence'], y=df_agg['id'], title=k, width=600, height=400).update_xaxes(categoryorder="total descending")
    fig.update_layout(xaxis_title="Confianza", yaxis_title="Cantidad de etiquetas")
    fig.show()

## 4) Tiempo total de etiquetado por persona
Sumar lead_time de todos los audios, por persona, por semana.

Me falta agregarlo a los df.

In [62]:
lead_time = {} # tiempo total por persona
for k in psicologos.keys():
    data = psicologos[k]
    l = [] # lead_time de cada audio
    for i in range(len(data)):
        l.append(data[i]['annotations'][0]['lead_time'])
    lead_time[k] = sum(l)/3600.0
    
fig = px.histogram(x=lead_time.keys(), y=lead_time.values(), title='Tiempo total de etiquetado por persona (en horas)', width=600, height=400).update_xaxes(categoryorder="total descending")
fig.update_layout(xaxis_title="Psicólogo", yaxis_title="Tiempo (h)")
fig.show()

## 5) Cantidad de updates por cada emoción

In [63]:
for k in psicologos.keys():
    df = df_dict[k]
    
    df_agg = df[['emotion', 'updated']].groupby(['emotion']).sum()
    df_agg.reset_index(inplace=True)

    fig = px.histogram(df_agg, x=df_agg['emotion'], y=df_agg['updated'], title=k, width=600, height=400)
    fig.show()

## Cuántos etiquetados por día hizo cada persona?

In [64]:
for k in psicologos.keys():
    df = df_dict[k]
    
    df_agg = df.groupby(['date']).count()
    df_agg.reset_index(inplace=True)

    fig = px.histogram(df_agg, x=df_agg['date'], y=df_agg['id'], title=k, width=600, height=400)
    fig.update_layout(xaxis_title="Fecha", yaxis_title="Cantidad de etiquetas")
    fig.show()

## Cantidad de etiquetas por cada emoción

In [66]:
for k in psicologos.keys():
    df = df_dict[k]

    fig = px.histogram(df, x="emotion", color="channel", title=k, width=800, height=600)
    fig.show()

## Cantidad por cada valor de intensidad

In [67]:
for k in psicologos.keys():
    df = df_dict[k]

    fig = px.histogram(df, x="intensity", color="channel", title=k, width=600, height=400)
    fig.show()

## Cantidad por cada valor de confianza

In [71]:
for k in psicologos.keys():
    df = df_dict[k]

    fig = px.histogram(df, x="confidence", color="channel", title=k, width=600, height=400)
    fig.show()

## Lead time

In [72]:
for k in psicologos.keys():
    df = df_dict[k]

    fig = px.histogram(df, x="lead_time", color="channel", title=k, width=600, height=400)
    fig.show()

## Tiempo entre create y update

In [75]:
for k in psicologos.keys():
    df = df_dict[k]

    fig = px.histogram(df, x="time_interval", color="channel", title=k, width=600, height=400)
    fig.show()